In [1]:
from openai import OpenAI
import shelve
from dotenv import load_dotenv
import os
import time
import pandas as pd
import json

C:\Users\pmair\AppData\Local\Temp\ipykernel_5192\1723992314.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
load_dotenv()
OPEN_AI_API_KEY = os.getenv("OPEN_AI_API_KEY")
client = OpenAI(api_key=OPEN_AI_API_KEY)

In [3]:
# this code might need optimization - very simple solution but works for now

# initialize thread dict
thread_dict = {
    "prompt": [],
    "answer": [],
    "user_id": [],
    "thread_id": [],
    "msg_id": [],
    "assistant_id": []  
}

# check if already threads exist
if os.path.exists("GPT_Threads\\threads.json"):
    # load exisiting thread
    with open('GPT_Threads\\threads.json') as json_file:
        data = json.load(json_file)  # this is a list of dictionaries
        for elem in data: 
            thread_dict["thread_id"].append(elem["thread_id"])
            thread_dict["msg_id"].append(elem["msg_id"])
            thread_dict["assistant_id"].append(elem["assistant_id"])
            thread_dict["prompt"].append(elem["prompt"])
            thread_dict["answer"].append(elem["answer"])
            thread_dict["user_id"].append(elem["user_id"])


In [12]:
def upload_file(path):
    file_ids = []
    # Upload a file with an "assistants" purpose
    #file = client.files.create(file=open(path, "rb"), purpose="assistants")
    for file_name in path:
        with open(file_name, "rb") as file_data:
            file_response = client.files.create(file=file_data, purpose='assistants')
            file_ids.append(file_response.id) 
    return file_ids

def create_assistant(file_ids):
    assistant = client.beta.assistants.create(
        name="Dashboard Onboarding ChatBot",
        instructions="I want you to be an assistant during a dashboard onboarding process. You should guide users of different expertise level step by step through a comprehensive journey of getting to know a PowerBI dashboard. The elements and layout of the dashboard are explained in the provided JSON file. I also provided you with previous user interactions in the form of csv files, which you should use to guide users to insightful states of the dashboard. Start the onboarding by asking the experience level of the new user and then give exercises based on this information. Beginner users should at least solve 5 exercises, showing them how to leverage PowerBI for data analysis, but also introduce them to the given dashboard. Only prompt one step at a time to the user, do not overwhelm him with too much information at once. Only give exercise you have knowledge about, hence you should be able to identify if the users answer is incorrect. Make sure to leverage the uploaded files",
        tools=[{"type": "retrieval"}, {"type": "code_interpreter"}],
        model="gpt-4-0125-preview",
        file_ids=file_ids,
    )
    return assistant

In [13]:
# Create an assistant with external knowledge
# only run once - check on website if assitant was created

# Upload file
files = upload_file(["Dashboard Files\encoding_initial_simplyfied.json", "Dashboard Files\merged_file.csv"])
# Create assistant
assistant = create_assistant(files)

In [14]:
assistant.id

'asst_KfZYtD7IwosiNbA8DBwTJ8Pp'

In [5]:
# Thread management
def check_if_thread_exists(user_id):
    with shelve.open("threads_db") as threads_shelf:
        return threads_shelf.get(user_id, None)

def store_thread(user_id, thread_id):
    with shelve.open("threads_db", writeback=True) as threads_shelf:
        threads_shelf[user_id] = thread_id

def save_thread(user_id, data_user, data_assistant):
    # saving threads in json file for better visualization of conversations
    thread_dict["thread_id"].append(data_assistant.thread_id)
    thread_dict["msg_id"].append(data_assistant.id)
    thread_dict["assistant_id"].append(data_assistant.assistant_id)
    thread_dict["prompt"].append(data_user.content[0].text.value)
    thread_dict["answer"].append(data_assistant.content[0].text.value)
    thread_dict["user_id"].append(user_id)

    df = pd.DataFrame(thread_dict)  # probably also a more neat solution without this step available
    df.to_json("GPT_Threads\\threads.json", orient="records", indent=2)

# Generate response
def generate_response(message_body, user_id, name):
    # Check if there is already a thread_id for the user_id
    thread_id = check_if_thread_exists(user_id)

    # If a thread doesn't exist, create one and store it
    if thread_id is None:
        print(f"Creating new thread for {name} with user_id {user_id}")
        thread = client.beta.threads.create()
        store_thread(user_id, thread.id)
        thread_id = thread.id

    # Otherwise, retrieve the existing thread
    else:
        print(f"Retrieving existing thread for {name} with user_id {user_id}")
        thread = client.beta.threads.retrieve(thread_id)

    # Add message to thread
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=message_body,
    )

    # Run the assistant and get the new message
    new_message = run_assistant(user_id, thread)
    print(f"To {name}:", new_message)
    return new_message

# Run assistant
def run_assistant(user_id, thread):
    # Retrieve the created assistant or paste the id of any other available assistant instead of "assistant.id"

    # Onboarding bot: asst_CzAZx0pbuCdy57fd18DFWZEB
    # Dashboard Onboarding Assistant: asst_irK6D1q8nwG8JTHN21ykPURB

    assistant = client.beta.assistants.retrieve(assistant.id)

    # Run the assistant
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )

    # Wait for completion
    while run.status != "completed":
        time.sleep(0.5)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    # Retrieve the Messages
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    new_message = messages.data[0].content[0].text.value

    data_assistant = client.beta.threads.messages.list(thread_id=thread.id).data[0]
    data_user = client.beta.threads.messages.list(thread_id=thread.id).data[1]

    save_thread(user_id, data_user, data_assistant)
    return new_message

In [19]:
# Test assistant
user_name = "Beginner"
user_id = "2"
new_message = generate_response("Start Dashboard onboarding", user_id, user_name)

Creating new thread for Beginner with user_id 2
To Beginner: Welcome to the dashboard onboarding process! To best cater to your specific needs, could you please share your experience level with PowerBI? You can choose among the following:

1. Beginner - You are new to PowerBI and might need guidance on basic functionalities and operations.
2. Intermediate - You have some experience with PowerBI, familiar with most functionalities but might need help with complex features.
3. Advanced - You are very comfortable with PowerBI, looking for expert tips or advanced functionalities.

Please select your experience level (1, 2, or 3):


In [20]:
new_message = generate_response("1", user_id, user_name)

Retrieving existing thread for Beginner with user_id 2
To Beginner: Great, as a beginner to PowerBI, we'll start with some basic concepts and functionalities to get you acquainted with the dashboard. 

Firstly, PowerBI is a business analytics service by Microsoft. It provides interactive visualizations and business intelligence capabilities with an interface simple enough for end users to create their own reports and dashboards.

Given that you're just starting, here's your first exercise:

**Exercise 1: Open a PowerBI Dashboard**
1. When you open PowerBI, you'll typically start at the Home view.
2. Look for the “Get Data” option on the bottom left corner of the screen to open an existing report or dashboard. For the purpose of this exercise, we'll pretend you're opening the dashboard provided in the JSON file I've received.
3. Once the dashboard is opened, familiarize yourself with the layout. Notice how visualizations are arranged on the canvas. Each chart, graph, or table you see is

In [21]:
new_message = generate_response("I see two summaries of new hire numbers, a comparison chart of two years. A line chart showing Full Time vs Part Time hires and 2 filter options. Did i miss something?", user_id, user_name)

Retrieving existing thread for Beginner with user_id 2
To Beginner: Based on the information in the JSON file, the components of the dashboard include:

1. **Two Card Visuals** for summarizing current values of New Hires and New Hires SPLY (Same Period Last Year), aligning well with the summaries you mentioned.
2. **Two Filters** for Region and Ethnicity, allowing for more granular examination of the data based on these specific attributes.
3. **A Line Chart** titled "New Hires by Month and FPDesc" that shows trends across different months and Full Time vs Part Time hires, matching your description of a line chart comparing Full Time vs Part Time.
4. **A Clustered Bar Chart** described as a "line clustered column combo chart" without a clear title provided in your summary but includes the insights for "New Hires by Region and Ethnicity". This could contribute to the comparison aspect you've noted.
5. Also mentioned is a **Line Clustered Column Combo Chart** for comparing "New Hires and

In [22]:
new_message = generate_response("I would expect the total number of new hires to go down, since we are narrowing the data. Also maybe new trends emerge if we set filters to specific regions", user_id, user_name)

Retrieving existing thread for Beginner with user_id 2
To Beginner: Exactly! Applying a filter for a specific region would indeed narrow down the data to new hires from that region, potentially reducing the total number of new hires reflected in the summaries and other visualizations like the "New Hires by Month and FPDesc" line chart. In addition, filtering by region might reveal new trends or insights specific to that region, such as whether certain months had unusually high or low hiring rates or if there's a significant difference in the Full-Time vs Part-Time hiring trends within that region.

By filtering, you're able to interactively analyze and understand the data from different perspectives, which is a powerful aspect of PowerBI dashboards. It allows users to drill down into specific areas of interest, uncovering insights that could be crucial for decision-making.

For your next beginner exercise, let's focus on another fundamental functionality:

**Exercise 3: Investigate Vis

In [23]:
new_message = generate_response("Selecting a specific region bar will apply the same filter as if I filtered by this region using the slicer I suppose", user_id, user_name)

Retrieving existing thread for Beginner with user_id 2
To Beginner: That's correct! Selecting a specific data point, such as a bar representing a particular region, essentially acts as an on-the-fly filter for that particular attribute. This type of interaction, known as "cross-filtering", allows the dashboard to dynamically update other visualizations based on the selection. 

Here's what typically happens when you perform such an action in a PowerBI dashboard:

- **Direct Filtering**: The selection directly filters the data across the dashboard to only display data relevant to the selected region. This means all metrics, charts, and tables that can be influenced by this selection will now only reflect the data for the chosen region.
- **Consistency Across Visualizations**: Other visualizations on the dashboard that are related to the selected attribute (in this case, region) will automatically adjust to reflect this narrower scope of data.
- **Highlight vs. Filter**: In some cases, i

In [24]:
new_message = generate_response("Which insights should I search for in this particular dashboard?", user_id, user_name)

Retrieving existing thread for Beginner with user_id 2
To Beginner: In exploring this specific PowerBI dashboard centered around new hires data, focusing on insights that drive strategic decisions or offer actionable intelligence is crucial. Here are several insights you might want to look for and how to potentially uncover them using the dashboard's functionalities:

1. **Trends Over Time**:
   - **Exercise**: Use the "New Hires by Month and FPDesc" line chart to observe how new hires (both Full Time and Part Time) have varied over different months.
   - **Insight**: Identifying patterns or trends, such as specific times of the year when hiring spikes or drops, can help in planning future recruitment efforts more effectively.

2. **Comparison Across Regions**:
   - **Exercise**: Utilize the "New Hires by Region and Ethnicity" bar chart along with the Region filter to compare hiring numbers across different regions.
   - **Insight**: Understanding which regions have higher or lower hir